# Description
这里使用TensorFlow 1.5实现用于分类任务的LR，数据集选用的是 MNIST。这是个手写数字数据库,包含60000个示例的训练集以及10000个示例的测试集.在 MNIST 数据集中的每张图片由 28 x 28 个像素点构成, 每个像素点用一个灰度值表示. 

In [2]:
import tensorflow as tf
# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)


Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
# Parameters
learning_rate = 0.01
training_epochs = 20
batch_size = 100
num_classes = 10
display_step = 1
feature_size = 28 * 28

In [4]:

tf.reset_default_graph()  # 重置网络结构
# tf Graph Input
# mnist data image of shape 28*28=784
input_x = tf.placeholder(dtype=tf.float32, shape=[None, 784], name="input_x")
# 0-9 digits recognition => 10 classes
input_y = tf.placeholder(dtype=tf.float32, shape=[None, 10], name="input_y")
# ------ v1 不调用接口 ------
# Set model weights
W = tf.Variable(tf.zeros([feature_size, num_classes]))  # shape=(784, 10)
b = tf.Variable(tf.zeros([num_classes]))  # shape=(10,)
# Construct model
pred = tf.nn.softmax(tf.matmul(input_x, W) + b)  # Softmax, shape=(?, 10)
# Minimize error using cross entropy
cost = tf.reduce_mean(-tf.reduce_sum(input_y *
                      tf.log(pred), reduction_indices=1))

# # ------ v2 调用接口 ------
# logits = tf.layers.dense(inputs=input_x, units=num_classes, use_bias=True,
#                          kernel_initializer=tf.zeros_initializer, name="LR")  # shape=(?, 10)
# pred = tf.nn.softmax(logits=logits, name="pred")  # shape=(?, 10)
# cost = tf.losses.softmax_cross_entropy(onehot_labels=input_y, logits=logits)

# Gradient Descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Test model
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(input_y, 1))
# Calculate accuracy for 3000 examples
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [5]:
# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()
# Start training
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Fit training using batch data
            _, c = sess.run([optimizer, cost], feed_dict={input_x: batch_xs,
                                                          input_y: batch_ys})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")
    
    print("Accuracy:", accuracy.eval(
        {input_x: mnist.test.images[:3000], input_y: mnist.test.labels[:3000]}))


Epoch: 0001 cost= 1.183992523
Epoch: 0002 cost= 0.665442883
Epoch: 0003 cost= 0.552889128
Epoch: 0004 cost= 0.498766734
Epoch: 0005 cost= 0.465416580
Epoch: 0006 cost= 0.442654295
Epoch: 0007 cost= 0.425560321
Epoch: 0008 cost= 0.412178602
Epoch: 0009 cost= 0.401371783
Epoch: 0010 cost= 0.392396411
Epoch: 0011 cost= 0.384758935
Epoch: 0012 cost= 0.378169611
Epoch: 0013 cost= 0.372395263
Epoch: 0014 cost= 0.367260074
Epoch: 0015 cost= 0.362720456
Epoch: 0016 cost= 0.358564532
Epoch: 0017 cost= 0.354887736
Epoch: 0018 cost= 0.351479514
Epoch: 0019 cost= 0.348264889
Epoch: 0020 cost= 0.345395626
Optimization Finished!
Accuracy: 0.88533336
